# Importing Libraries

In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

In [103]:
# Retrieving stored variables
%store -r ls_with_img
%store -r specimen_label_dict
%store -r df

In [104]:
df.head()

,id,institutionCode,collectionCode,ownerInstitutionCode,basisOfRecord,occurrenceID,catalogNumber,otherCatalogNumbers,higherClassification,kingdom,...,verbatimDepth,verbatimElevation,disposition,language,recordEnteredBy,modified,sourcePrimaryKey-dbpk,collID,recordID,references
0,91069,NMCR,NaN,NaN,PreservedSpecimen,NaN,9551,NaN,Plantae|Spermatophyta|Tracheophyta|Magnoliophy...,Plantae,...,NaN,ft.,NaN,NaN,NaN,1991-01-28 00:00:00,9551,23,1d8086ed-5918-465f-8b89-a75dcb4dbbf3,https://intermountainbiota.org/portal/collecti...
1,91079,NMCR,NaN,NaN,PreservedSpecimen,NaN,10353,NaN,Plantae|Spermatophyta|Tracheophyta|Magnoliophy...,Plantae,...,NaN,ft.,NaN,NaN,NaN,1989-09-28 00:00:00,10353,23,372d33ef-0d0a-4f8d-8013-afb593df7c93,https://intermountainbiota.org/portal/collecti...
2,101418,ENMU,NaN,NaN,PreservedSpecimen,1b95e7a4-46a6-4d71-bd98-ab14acc062e1,3789,NaN,Plantae|Spermatophyta|Tracheophyta|Magnoliophy...,Plantae,...,NaN,4500ft.,NaN,NaN,NaN,2003-08-19 01:00:00,3789,24,1b95e7a4-46a6-4d71-bd98-ab14acc062e1,https://intermountainbiota.org/portal/collecti...
3,101419,ENMU,NaN,NaN,PreservedSpecimen,89363aae-119b-4c9f-8c4b-b76c59eaf01b,3790,NaN,Plantae|Spermatophyta|Tracheophyta|Magnoliophy...,Plantae,...,NaN,4500ft.,NaN,NaN,NaN,2003-08-19 01:00:00,3790,24,89363aae-119b-4c9f-8c4b-b76c59eaf01b,https://intermountainbiota.org/portal/collecti...
4,101420,ENMU,NaN,NaN,PreservedSpecimen,2135e2b4-d1a5-4688-a760-b9160d10ab2e,3791,NaN,Plantae|Spermatophyta|Tracheophyta|Magnoliophy...,Plantae,...,NaN,NaN,NaN,NaN,NaN,2003-08-19 01:00:00,3791,24,2135e2b4-d1a5-4688-a760-b9160d10ab2e,https://intermountainbiota.org/portal/collecti...


# Importing Pre-trained Model(s)
  - As per standard Transfer learning methodology, we'll be using pre-trained model(s) to train on our dataset. To import our models, we'll use Tensorflows API, Keras. Keras is an open-source library that happens to provide a wide range of deep learning models that we can use at our convenience: https://keras.io/api/applications/

In [105]:
import tensorflow as tf
from keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications.vgg16 import preprocess_input

In [106]:
vgg16_model = Sequential()

pretrained_vgg16 = tf.keras.applications.VGG16(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',  # Global average pooling
                   weights='imagenet')

for layer in pretrained_vgg16.layers:
    layer.trainable = False

vgg16_model.add(pretrained_vgg16)
vgg16_model.add(Flatten())
vgg16_model.add(Dense(512, activation='relu'))
vgg16_model.add(Dense(4, activation='softmax'))  

vgg16_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten_6 (Flatten)         (None, 512)               0         
                                                                 
 dense_12 (Dense)            (None, 512)               262656    
                                                                 
 dense_13 (Dense)            (None, 4)                 2052      
                                                                 
Total params: 14,979,396
Trainable params: 264,708
Non-trainable params: 14,714,688
_________________________________________________________________


In [107]:
vgg16_model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [108]:
image_dir = '/mnt/c/users/John/Desktop/Capstone Project Files/Capstone Project Data/images'
# specimen_label_dict

In [109]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224)) 
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img) 
    return img

In [110]:
image_filenames = [os.path.join(image_dir, f"{key}.jpg") for key in specimen_label_dict.keys()]

In [111]:
def image_batch_generator(batch_size):
    while True:
        batch_paths = np.random.choice(a=image_filenames, size=batch_size)
        batch_input = []
        batch_output = []

        for image_path in batch_paths:
            base_name = os.path.splitext(os.path.basename(image_path))[0]

            if base_name not in specimen_label_dict:
                print(f"Key not found in dictionary: {base_name}")
                continue  # Skip this image

            image = preprocess_image(image_path)
            label = specimen_label_dict[base_name]

            batch_input.append(image)
            batch_output.append(label)

        batch_x = np.array(batch_input)
        batch_y = np.array(batch_output)

        yield (batch_x, batch_y)

In [112]:
batch_size = 16

generator = image_batch_generator(batch_size)

In [113]:
# Training our model
vgg16_model.fit(
    generator,
    steps_per_epoch=len(image_filenames) // batch_size,
    epochs=10  
)

Key not found in dictionary: 20227480
Key not found in dictionary: 723001
Key not found in dictionary: 8079019
Key not found in dictionary: 24479429
Key not found in dictionary: 24164359
Key not found in dictionary: 26967734
Key not found in dictionary: 7950818
Key not found in dictionary: 7950696
Key not found in dictionary: 29466238
Key not found in dictionary: 28728436
Key not found in dictionary: 13869713
Key not found in dictionary: 2124571
Key not found in dictionary: 1097255
Key not found in dictionary: 19632582
Key not found in dictionary: 27510554
Key not found in dictionary: 13867790
Epoch 1/10


ValueError: in user code:

    File "/home/johnbaguinaga/.local/lib/python3.7/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/johnbaguinaga/.local/lib/python3.7/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/johnbaguinaga/.local/lib/python3.7/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/johnbaguinaga/.local/lib/python3.7/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/home/johnbaguinaga/.local/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/johnbaguinaga/.local/lib/python3.7/site-packages/keras/engine/input_spec.py", line 251, in assert